Есть дамп данных о всех НКО в России, содержится информация о получении государственных грантов, госконтрактов и субсидий, регионе и дате регистрации и ОКВЭД (классификатор экономической деятельности).

- Проверить, есть ли зависимость вероятности получения грантов от государства / госконтрактов от региона регистрации организации
- Проверить, есть ли зависимость вероятности получения грантов от государства / госконтрактов от возраста организации
- Проверить, есть ли зависимость вероятности получения грантов от государства / госконтрактов от экономической деятельности
- Любые дополнительные выводы и проверки

In [463]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # библиотека визуализации
from scipy import stats # библиотека для расчетов
from ast import literal_eval as le
import math

In [464]:
# читаем все файлы
data1 = pd.read_csv('data/ngo_dump_01.csv')
data2 = pd.read_csv('data/ngo_dump_02.csv')
data3 = pd.read_csv('data/ngo_dump_03.csv')
data4 = pd.read_csv('data/ngo_dump_04.csv')
data5 = pd.read_csv('data/ngo_dump_05.csv')
data6 = pd.read_csv('data/ngo_dump_06.csv')
data7 = pd.read_csv('data/ngo_dump_07.csv')
data8 = pd.read_csv('data/ngo_dump_08.csv')
data9 = pd.read_csv('data/ngo_dump_09.csv')
data10 = pd.read_csv('data/ngo_dump_10.csv')
data11 = pd.read_csv('data/ngo_dump_11.csv')
data12 = pd.read_csv('data/ngo_dump_12.csv')
data13 = pd.read_csv('data/ngo_dump_13.csv')
data14 = pd.read_csv('data/ngo_dump_14.csv')
data15 = pd.read_csv('data/ngo_dump_15.csv')

# объединяем в одну таблицу
data = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14,data15])
pd.set_option('display.max_columns', None)
data.info()

/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/207446370.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv('data/ngo_dump_01.csv')
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/207446370.py:5: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data4 = pd.read_csv('data/ngo_dump_04.csv')
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/207446370.py:7: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data6 = pd.read_csv('data/ngo_dump_06.csv')
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/207446370.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data9 = pd.read_csv('data/ngo_dump_09.csv')
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/207446370.py:11:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747034 entries, 0 to 49999
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   minjustRegNum       462067 non-null  object 
 1   regionName          747009 non-null  object 
 2   logo                0 non-null       float64
 3   statusDetail        747034 non-null  object 
 4   fullName            747034 non-null  object 
 5   dateReg             321416 non-null  object 
 6   minjustForm         461073 non-null  object 
 7   charter             0 non-null       float64
 8   minjustStatus       462302 non-null  object 
 9   ogrn                747034 non-null  int64  
 10  opf                 747034 non-null  object 
 11  oktmo               747034 non-null  object 
 12  egrulStatus         747034 non-null  object 
 13  mainOkved           747034 non-null  object 
 14  regionCode          747034 non-null  int64  
 15  incomeTotal         747034 non-null

In [465]:
# оставляем только нужные столбцы
df = data[['ogrn', 'fullName', 'regionName', 'mainOkved', 'addOkved', 'dateReg', 'dateOgrn','dateLiquid', 'incomeTotal', 'incomeDetail']]
df


,ogrn,fullName,regionName,mainOkved,addOkved,dateReg,dateOgrn,dateLiquid,incomeTotal,incomeDetail
0,1207800141790,МЕЖРЕГИОНАЛЬНАЯ АНТРОПОСОФСКАЯ МЕДИЦИНСКАЯ АСС...,Санкт-Петербург,{'name': 'Деятельность профессиональных членск...,[],NaN,{'$date': '2020-10-30T00:00:00.000Z'},NaN,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
1,1207700349152,"ФОНД ""СОДЕЙСТВИЕ ДУХОВНОМУ РАЗВИТИЮ ЧЕЛОВЕКА ""...",Москва,{'name': 'Предоставление прочих финансовых усл...,"[{'name': 'Издание книг', 'code': '58.11', 've...",NaN,{'$date': '2020-09-23T00:00:00.000Z'},NaN,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
2,1207800011430,"КОЛЛЕГИЯ АДВОКАТОВ ""САНКТ-ПЕТЕРБУРГСКАЯ КОЛЛЕГ...",Санкт-Петербург,"{'name': 'Деятельность в области права', 'code...",[],NaN,{'$date': '2020-02-03T00:00:00.000Z'},NaN,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
3,1207700348976,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ НЕДВИЖИМОСТИ ""ЛУГИН...",Москва,{'name': 'Управление недвижимым имуществом за ...,[{'name': 'Аренда и управление собственным или...,NaN,{'$date': '2020-09-23T00:00:00.000Z'},NaN,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
4,1207800081500,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР С...",Санкт-Петербург,{'name': 'Деятельность по организации конферен...,[{'name': 'Исследование конъюнктуры рынка и из...,NaN,{'$date': '2020-07-08T00:00:00.000Z'},NaN,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
...,...,...,...,...,...,...,...,...,...,...
49995,1067746424855,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ ЖИЛЬЯ ""БОЛЬШОЙ ХАРИ...",Москва,"{'name': 'Управление недвижимым имуществом', '...",[{'name': 'Управление эксплуатацией жилого фон...,NaN,{'$date': '2006-03-29T00:00:00.000Z'},{'$date': '2009-01-12T00:00:00.000Z'},0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
49996,1069600012460,НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ МЕЖРЕГИОНАЛЬНЫЙ ФОН...,Свердловская область,{'name': 'Деятельность по изучению общественно...,"[{'name': 'Деятельность в области права', 'cod...",NaN,{'$date': '2006-08-09T00:00:00.000Z'},{'$date': '2012-08-15T00:00:00.000Z'},0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
49997,1068602157447,"СОВЕТ САМОУПРАВЛЕНИЯ МИКРОРАЙОНА ""А""",Ханты-Мансийский автономный округ - Югра,"{'name': None, 'code': None, 'version': 'ОК 02...",[],{'$date': '1998-01-01T00:00:00.000Z'},{'$date': '2006-10-11T00:00:00.000Z'},{'$date': '2006-10-11T00:00:00.000Z'},0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."
49998,1064202020343,КООПЕРАТИВ РИТУАЛ ПО ОКАЗАНИЮ РИТУАЛЬНЫХ УСЛУГ...,Кемеровская область,"{'name': None, 'code': None, 'version': 'ОК 02...",[],{'$date': '1987-12-05T00:00:00.000Z'},{'$date': '2006-06-05T00:00:00.000Z'},{'$date': '2006-06-05T00:00:00.000Z'},0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '..."


In [466]:
# фильтруем названия организаций
# 'БЛАГОТВОРИТ', но без “живот”, “зоо”, “собак”, “кошк”, “сельхоз”, “сельскохоз”
# df['fullName'] = df['fullName'].apply(lambda s:str(s).upper())
# mask1 = df['fullName'].str.contains('БЛАГОТВОРИТ')
# mask2 = ~((df['fullName'].str.contains('ЖИВОТ')) & (df['fullName'].str.contains('ЗОО')) & (df['fullName'].str.contains('СОБАК')))
# mask3 = ~((df['fullName'].str.contains('КОШК')) & (df['fullName'].str.contains('СЕЛЬХОЗ')) & (df['fullName'].str.contains('СЕЛЬСКОХОЗ')))

# df = df[mask1 & mask2 & mask3]
# df.head()


# фильтруем названия организаций
# 'БЛАГОТВОРИТ', но без “живот”, “зоо”, “собак”, “кошк”, “сельхоз”, “сельскохоз”
df['fullName']=df['fullName'].apply(lambda s:str(s).upper())
df['fond']=df['fullName'].apply(lambda x: 1 if x.find('БЛАГОТВОРИТ')>1 else 0 )
df['no_fond']=df['fullName'].apply(lambda x: 1 if x.find('ЖИВОТ' or 'ЗОО' or 'СОБАК' or 'СЕЛЬХОЗ' or 'СЕЛЬСКОХОЗ' or 'КОШК' )>1 else 0 )
df=df[(df['fond']== 1) & (df['no_fond'] == 0)]
df.info()

/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/3391178.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fullName']=df['fullName'].apply(lambda s:str(s).upper())
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/3391178.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fond']=df['fullName'].apply(lambda x: 1 if x.find('БЛАГОТВОРИТ')>1 else 0 )


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12694 entries, 5 to 49965
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ogrn          12694 non-null  int64  
 1   fullName      12694 non-null  object 
 2   regionName    12694 non-null  object 
 3   mainOkved     12694 non-null  object 
 4   addOkved      12694 non-null  object 
 5   dateReg       5964 non-null   object 
 6   dateOgrn      12694 non-null  object 
 7   dateLiquid    8818 non-null   object 
 8   incomeTotal   12694 non-null  float64
 9   incomeDetail  12694 non-null  object 
 10  fond          12694 non-null  int64  
 11  no_fond       12694 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.3+ MB


/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/3391178.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['no_fond']=df['fullName'].apply(lambda x: 1 if x.find('ЖИВОТ' or 'ЗОО' or 'СОБАК' or 'СЕЛЬХОЗ' or 'СЕЛЬСКОХОЗ' or 'КОШК' )>1 else 0 )


In [467]:
# преобразуем даты - вычислим возраст организаций в месяцах
# если есть дата ликвидации: age  = dateLiquid - dateReg
# если есть нет даты ликвидации: age = 26.08.21 - dateReg
# 
# если есть dateReg - использовать его и считать приоритетом
# в остальных случаях использовать dateOgrn
def to_date(row):
    if row['is_reg']:
        return row['dateOgrn']
    else:
        return row['dateReg']

# вспомогательный столбец - маячок, есть ли дата dateReg
df['is_reg'] = df['dateReg'].isnull()
# формирование вспомогательного столбца с датой(dateReg или dateOgrn), обрезка строки и перевод в формат даты
res = df.apply(to_date,axis=1)
res = res.apply(lambda x: x[11:-16])
df['date'] = pd.to_datetime(res)

# перевод столбца dateLiquid в формат даты
# пропущенные значения заполняем датой 2021-08-26 (дата актуальности данных)
res = df['dateLiquid'].fillna('2021-08-26')
res = res.apply(lambda x: x[11:-16] if x!='2021-08-26' else x)
df['dateLiquid'] = pd.to_datetime(res)

# считаем возраст и добавляем столбец age
# возраст в днях

df['age'] = round((df['dateLiquid'] - df['date'])/ np.timedelta64(1,"Y"),0) # 


# удаляем вспомогательные столбцы, dateLiquid, dateReg, dateOgrn
df = df.drop(['is_reg','date','dateLiquid', 'dateReg', 'dateOgrn'], axis=1)
df

/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/1359134596.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_reg'] = df['dateReg'].isnull()
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/1359134596.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(res)
/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/ipykernel_13960/1359134596.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,ogrn,fullName,regionName,mainOkved,addOkved,incomeTotal,incomeDetail,fond,no_fond,age
5,1207700350440,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР К...",Москва,{'name': 'Предоставление прочих социальных усл...,"[{'name': 'Печатание газет', 'code': '18.11', ...",0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0
154,1207700405637,"МЕЖДУНАРОДНЫЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""МИЛОСЕРД...",Москва,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Предоставление прочих финансовых ус...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0
341,1204700005971,РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБЩЕСТВЕННАЯ ОР...,Ленинградская область,{'name': 'Деятельность прочих общественных орг...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0
532,1206100022810,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ БЛАГОТВОРИТЕЛЬНАЯ ОР...,Ростовская область,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Деятельность прочих общественных ор...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0
574,1203300003928,ВЛАДИМИРСКАЯ РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБ...,Владимирская область,{'name': 'Деятельность прочих общественных орг...,[{'name': 'Деятельность по изучению общественн...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0
...,...,...,...,...,...,...,...,...,...,...
49740,1067799018715,"НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""БЛАГОТВОРИТЕЛЬНЫЙ ...",Москва,"{'name': 'Прочее финансовое посредничество', '...",[{'name': 'Предоставление социальных услуг без...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,9.0
49786,1067424003624,"ДЕТСКИЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""РАДУГА""",Челябинская область,{'name': 'Предоставление социальных услуг без ...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,6.0
49877,1067799022752,НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ БЛАГОТВОРИТЕЛЬНЫЙ Ф...,Москва,{'name': 'Научные исследования и разработки в ...,[{'name': 'Научные исследования и разработки в...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,2.0
49957,1066900006975,"ТВЕРСКОЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""НАДЕЖДА""",Тверская область,{'name': 'Предоставление социальных услуг без ...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,9.0


In [468]:

# вытаскиваем из mainOkved название деятельности и записываем это в новый столбец mainOkved_name
def okved(x):
    result = le(x)
    return result['name']
df['mainOkved_name'] = df['mainOkved'].apply(okved)


In [469]:
# вытаскиваем из mainOkved код деятельности и записываем это в новый столбец mainOkved_code
def okved_code(x):
   result = le(x)
   return result['code']
df['mainOkved_code'] = df['mainOkved'].apply(okved_code)

In [490]:
le(df['addOkved'][5])[2]['name']

'Издание журналов и периодических изданий'

In [488]:
# вытаскиваем из addOkved список названий деятельности и записываем это в новый столбец add_okved_name
def add_okved(x):
    res = []
    x = x[1:-1].split('},') #делим входную строку на список строк-словарей
    for i in range(len(x)-1):
        s = x[i]+'}'
        s = le(s)
        res.append(s['name'])
    if res!=[]:
        i = len(x)-1
        res.append(le(x[i])['name'])
    return res


le(df['addOkved'][1])[2]['name']

len(le(data_1['addOkved'][1]))-это для циклов

df['add_okved_name'] = df['addOkved'].apply(add_okved)

# вытаскиваем из addOkved список кодов деятельности и записываем это в новый столбец add_okved_code
def add_okved_code(x):
    res = []
    x = x[1:-1].split('},')
    for i in range(len(x)-1):
        s = x[i]+'}'
        s = le(s)
        res.append(s['code'])
    if res!=[]:
        s=le(x[-1])
        res.append(s['code'])
    return res

df['add_Okved_code'] = df['addOkved'].apply(add_okved_code)
df


,ogrn,fullName,regionName,mainOkved,addOkved,incomeTotal,incomeDetail,fond,no_fond,age,mainOkved_name,mainOkved_code,add_okved_name,add_Okved_code
5,1207700350440,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР К...",Москва,{'name': 'Предоставление прочих социальных усл...,"[{'name': 'Печатание газет', 'code': '18.11', ...",0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,Предоставление прочих социальных услуг без обе...,88.99,"[Печатание газет, Издание книг, Издание журнал...","[18.11, 58.11, 58.14, 58.19, 93.29.9, 88.10, 8..."
154,1207700405637,"МЕЖДУНАРОДНЫЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""МИЛОСЕРД...",Москва,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Предоставление прочих финансовых ус...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,Предоставление прочих социальных услуг без обе...,88.99,"[Предоставление прочих финансовых услуг, кроме...","[64.99, 58.19, 93.29.9, 72.19, 90.01, 88.10, 8..."
341,1204700005971,РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБЩЕСТВЕННАЯ ОР...,Ленинградская область,{'name': 'Деятельность прочих общественных орг...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,"Деятельность прочих общественных организаций, ...",94.99,[],[]
532,1206100022810,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ БЛАГОТВОРИТЕЛЬНАЯ ОР...,Ростовская область,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Деятельность прочих общественных ор...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,Предоставление прочих социальных услуг без обе...,88.99,"[Деятельность прочих общественных организаций,...","[94.99, 88.10]"
574,1203300003928,ВЛАДИМИРСКАЯ РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБ...,Владимирская область,{'name': 'Деятельность прочих общественных орг...,[{'name': 'Деятельность по изучению общественн...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,"Деятельность прочих общественных организаций, ...",94.99,[Деятельность по изучению общественного мнения...,"[73.20.2, 93.29, 82.30, 88.99]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49740,1067799018715,"НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""БЛАГОТВОРИТЕЛЬНЫЙ ...",Москва,"{'name': 'Прочее финансовое посредничество', '...",[{'name': 'Предоставление социальных услуг без...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,9.0,Прочее финансовое посредничество,65.2,[],[]
49786,1067424003624,"ДЕТСКИЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""РАДУГА""",Челябинская область,{'name': 'Предоставление социальных услуг без ...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,6.0,Предоставление социальных услуг без обеспечени...,85.32,[],[]
49877,1067799022752,НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ БЛАГОТВОРИТЕЛЬНЫЙ Ф...,Москва,{'name': 'Научные исследования и разработки в ...,[{'name': 'Научные исследования и разработки в...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,2.0,Научные исследования и разработки в области ес...,73.1,[Научные исследования и разработки в области о...,"[73.20, 22.1, 91.1]"
49957,1066900006975,"ТВЕРСКОЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""НАДЕЖДА""",Тверская область,{'name': 'Предоставление социальных услуг без ...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,9.0,Предоставление социальных услуг без обеспечени...,85.32,[],[]


In [ ]:
# вытаскиваем гранты из incomeDetail
def grants_count(x):
    result = le(x)
    return result['grants']['totalCount']

def grants_sum(x):
    result = le(x)
    return result['grants']['totalSum']

# вытаскиваем субсидии из incomeDetail
def fedSubsidies_count(x):
    result = le(x)
    return result['fedSubsidies']['totalCount']

def fedSubsidies_sum(x):
    result = le(x)
    return result['fedSubsidies']['totalSum']

# вытаскиваем все контракты из incomeDetail
def contracts_count(x):
    result = le(x)
    count = result['contracts44']['totalCount'] + result['contracts223']['totalCount'] + result['contracts94']['totalCount']
    return count

def contracts_sum(x):
    result = le(x)
    sum = result['contracts44']['totalSum'] + result['contracts223']['totalSum'] + result['contracts94']['totalSum']
    return sum


df['grants_count'] = df['incomeDetail'].apply(grants_count) #количество грантов
df['grants_sum'] = df['incomeDetail'].apply(grants_sum) #сумма грантов

df['fedSubsidies_count'] = df['incomeDetail'].apply(fedSubsidies_count) #количество субсидий
df['fedSubsidies_sum'] = df['incomeDetail'].apply(fedSubsidies_sum) #сумма субсидий

df['contracts_count'] = df['incomeDetail'].apply(contracts_count) #количество контрактов
df['contracts_sum'] = df['incomeDetail'].apply(contracts_sum) #сумма контрактов

df.head()

,ogrn,fullName,regionName,mainOkved,addOkved,incomeTotal,incomeDetail,fond,no_fond,age,mainOkved_name,grants_count,grants_sum,fedSubsidies_count,fedSubsidies_sum,contracts_count,contracts_sum
5,1207700350440,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР К...",Москва,{'name': 'Предоставление прочих социальных усл...,"[{'name': 'Печатание газет', 'code': '18.11', ...",0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,Предоставление прочих социальных услуг без обе...,0,0.0,0,0.0,0,0.0
154,1207700405637,"МЕЖДУНАРОДНЫЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""МИЛОСЕРД...",Москва,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Предоставление прочих финансовых ус...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,Предоставление прочих социальных услуг без обе...,0,0.0,0,0.0,0,0.0
341,1204700005971,РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБЩЕСТВЕННАЯ ОР...,Ленинградская область,{'name': 'Деятельность прочих общественных орг...,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,"Деятельность прочих общественных организаций, ...",0,0.0,0,0.0,0,0.0
532,1206100022810,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ БЛАГОТВОРИТЕЛЬНАЯ ОР...,Ростовская область,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Деятельность прочих общественных ор...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,Предоставление прочих социальных услуг без обе...,0,0.0,0,0.0,0,0.0
574,1203300003928,ВЛАДИМИРСКАЯ РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБ...,Владимирская область,{'name': 'Деятельность прочих общественных орг...,[{'name': 'Деятельность по изучению общественн...,0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",1,0,1.0,"Деятельность прочих общественных организаций, ...",0,0.0,0,0.0,0,0.0


In [ ]:
df = df.drop(['incomeDetail'], axis=1)
df = df.drop(['fond'], axis=1)
df = df.drop(['no_fond'], axis=1)

In [ ]:
df

,ogrn,fullName,regionName,mainOkved,addOkved,incomeTotal,age,mainOkved_name,grants_count,grants_sum,fedSubsidies_count,fedSubsidies_sum,contracts_count,contracts_sum
5,1207700350440,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР К...",Москва,{'name': 'Предоставление прочих социальных усл...,"[{'name': 'Печатание газет', 'code': '18.11', ...",0.0,1.0,Предоставление прочих социальных услуг без обе...,0,0.0,0,0.0,0,0.0
154,1207700405637,"МЕЖДУНАРОДНЫЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""МИЛОСЕРД...",Москва,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Предоставление прочих финансовых ус...,0.0,1.0,Предоставление прочих социальных услуг без обе...,0,0.0,0,0.0,0,0.0
341,1204700005971,РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБЩЕСТВЕННАЯ ОР...,Ленинградская область,{'name': 'Деятельность прочих общественных орг...,[],0.0,1.0,"Деятельность прочих общественных организаций, ...",0,0.0,0,0.0,0,0.0
532,1206100022810,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ БЛАГОТВОРИТЕЛЬНАЯ ОР...,Ростовская область,{'name': 'Предоставление прочих социальных усл...,[{'name': 'Деятельность прочих общественных ор...,0.0,1.0,Предоставление прочих социальных услуг без обе...,0,0.0,0,0.0,0,0.0
574,1203300003928,ВЛАДИМИРСКАЯ РЕГИОНАЛЬНАЯ БЛАГОТВОРИТЕЛЬНАЯ ОБ...,Владимирская область,{'name': 'Деятельность прочих общественных орг...,[{'name': 'Деятельность по изучению общественн...,0.0,1.0,"Деятельность прочих общественных организаций, ...",0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49740,1067799018715,"НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""БЛАГОТВОРИТЕЛЬНЫЙ ...",Москва,"{'name': 'Прочее финансовое посредничество', '...",[{'name': 'Предоставление социальных услуг без...,0.0,9.0,Прочее финансовое посредничество,0,0.0,0,0.0,0,0.0
49786,1067424003624,"ДЕТСКИЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""РАДУГА""",Челябинская область,{'name': 'Предоставление социальных услуг без ...,[],0.0,6.0,Предоставление социальных услуг без обеспечени...,0,0.0,0,0.0,0,0.0
49877,1067799022752,НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ БЛАГОТВОРИТЕЛЬНЫЙ Ф...,Москва,{'name': 'Научные исследования и разработки в ...,[{'name': 'Научные исследования и разработки в...,0.0,2.0,Научные исследования и разработки в области ес...,0,0.0,0,0.0,0,0.0
49957,1066900006975,"ТВЕРСКОЙ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""НАДЕЖДА""",Тверская область,{'name': 'Предоставление социальных услуг без ...,[],0.0,9.0,Предоставление социальных услуг без обеспечени...,0,0.0,0,0.0,0,0.0


In [ ]:
# проверка на дубликаты

mask = df.duplicated()
df_duplicates = df[mask]
print(f'Число найденных дубликатов: {df_duplicates.shape[0]}')

Число найденных дубликатов: 0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12694 entries, 5 to 49965
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ogrn                12694 non-null  int64  
 1   fullName            12694 non-null  object 
 2   regionName          12694 non-null  object 
 3   mainOkved           12694 non-null  object 
 4   addOkved            12694 non-null  object 
 5   incomeTotal         12694 non-null  float64
 6   age                 12694 non-null  float64
 7   mainOkved_name      7279 non-null   object 
 8   grants_count        12694 non-null  int64  
 9   grants_sum          12694 non-null  float64
 10  fedSubsidies_count  12694 non-null  int64  
 11  fedSubsidies_sum    12694 non-null  float64
 12  contracts_count     12694 non-null  int64  
 13  contracts_sum       12694 non-null  float64
 14  mainOkved_code      7279 non-null   object 
dtypes: float64(5), int64(4), object(6)
memory usage: 1.5+

In [ ]:
# провераяем на выбросы - смотрим описание таблицы
df.describe()

,ogrn,incomeTotal,age,grants_count,grants_sum,fedSubsidies_count,fedSubsidies_sum,contracts_count,contracts_sum
count,1.269400e+04,1.269400e+04,12694.000000,12694.000000,1.269400e+04,12694.000000,1.269400e+04,12694.000000,1.269400e+04
mean,1.076139e+12,5.337356e+05,11.044746,0.081928,2.022520e+05,0.009059,2.000653e+05,0.168741,1.314184e+05
std,1.415626e+11,1.756263e+07,6.294717,0.510876,1.810680e+06,0.190361,1.699935e+07,5.839907,3.407633e+06
min,1.020100e+12,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,1.034200e+12,0.000000e+00,6.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,1.053804e+12,0.000000e+00,10.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
75%,1.102000e+12,0.000000e+00,16.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
max,5.177746e+12,1.899607e+09,41.000000,9.000000,8.046482e+07,9.000000,1.899607e+09,525.000000,2.748680e+08
